This notebook will be used as a base and support in the implementation of the nlpaug library for text augmentation.


Inspiration taken from : https://www.kaggle.com/code/andypenrose/text-augmentation-with-nlpaug

In [1]:
%load_ext autoreload
%autoreload 2

## Dataset

In [2]:
import pandas as pd

df = pd.read_csv("../../data/silver/df_cleantext_v0.csv")
df

,Category,Message
0,0,go until jurong point crazy available only in ...
1,0,ok lar joking wif u oni
2,1,free entry in number a wkly comp to win fa cup...
3,0,u dun say so early hor u c already then say
4,0,nah i dont think he goes to usf he lives aroun...
...,...,...
5152,1,this is the numbernd time we have tried number...
5153,0,will you b going to esplanade fr home
5154,0,pity was in mood for that soany other suggestions
5155,0,the guy did some bitching but i acted like id ...


In [3]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.word.context_word_embs as nawcwe
import nlpaug.augmenter.word.word_embs as nawwe
import nlpaug.augmenter.word.spelling as naws

In [4]:
import os
import sys
sys.path.append(os.path.abspath("../utils"))
from experiments_utils import print_and_highlight_diff

## Chunk for analysis

In [5]:
pd.set_option('display.max_colwidth', None)

sample = df['Message'].iloc[:5].astype(str).to_list()


## KeyboardAug method

In [6]:
aug = nac.KeyboardAug()
augmented_texts = aug.augment(sample)

print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
go until jurong pling cTaz^ avxilaf>e on:T in bugis n great world la e buffet cine tTege got Xmire wat 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
ok lar jok&hg wif u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
b5ee entry in njmbed a w.<y cojO to win fa cup vinaI tkts n*mb3rs5 may number tsst fa to number to recF(v$ entry questionstd txt ratetcs zp9ly numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 11
u dun say so eW#ly hor u c al#eSdg fheb say 
----------------------

Notes: 
1. The original criteria decided to not focus on special chars nor numbers so the cleaning acted accordingly. These two params should be changed. Same for upper/lower chars. 
2. Many characters of each modified word are changed and the result doesn't look realistic imo. This augmenter technique creates unrealistic results.


In [7]:
aug = nac.KeyboardAug(aug_word_p=0.1, include_numeric=False, include_special_char=False, include_upper_case=False)
augmented_texts = aug.augment(sample)

print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
go uhtol jurogn point crazy available only in bugis n great world la e buffet cine there got amore wat 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
ok lar iokong wif u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
free entry in number a sjly comp to win fa cup final tkts numberst may number text fa to njjber to receive entry questionstd txt ratetcs apply mkmgeekverjumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 11
u dun say so exrlg hor u c already ghwn say 
----------------------

Notes: 

The misspellings are very hardcore and artificial. They don't make sense to me. 


In [8]:
# help(nac.KeyboardAug)
# help(aug.augment)

## SpellingAug method

This method substitutes word by spelling mistake words dictionary

In [9]:
aug = naws.SpellingAug()
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
go until jurong point crazy available only in bugis On gteat world a e buffe cine there gate. amore way 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
OK)][[... lar joking wif You oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
fee entry lin nummber g wkly comp ro win fa cup finel tkts numberst May number tex fa ou number to receive entry questionstd txt ratetcs apply numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 11
u dun say sooo eary hor You c already then said 
--------

Notes: 

This method creates more realistic results than the previous technique.

In [16]:
aug = naws.SpellingAug(aug_p = 0.5)
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
goes until jurong opint creasy available only ing bugis n greats worls a a buffet cinema where got amore wat 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
of lar chocking wif You oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
free entry ia namber's de wkly comp ty win fa cop finel tkts numberst mays namber text fa to number tu receive entry questionstd txt ratetcs Aply numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 11
You dun say saw early hot You c alread then saying 
--

Notes:

1. The results are way more realistic.
2. The dictionary includes upper case that should be treated later on.    


In [ ]:
#help(naws.SpellingAug())
# help(aug.augment)

## SynonymAug

Substitute similar word according to WordNet/ PPDB synonym

In [23]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/maldu/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [24]:
aug = naw.SynonymAug()
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 22
move until jurong item crazy available only in bugis n gravid humans atomic number 57 e buffet cine there got amore wat 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
o.k. lar joking wif u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
free entry in number a wkly comp to win fa cup concluding tkts numberst may number text fa to number to get entranceway questionstd txt ratetcs apply numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 12
u dun say so other hor u ampere second al

Notes:

- Original: 11
- u dun say so early hor u c already then say
- Augmented: 12
- u dun say so other hor u ampere second already then say 

some add extra words and I don't understand why

In [27]:
help(naw.SynonymAug())

Help on SynonymAug in module nlpaug.augmenter.word.synonym object:

class SynonymAug(nlpaug.augmenter.word.word_augmenter.WordAugmenter)
 |  SynonymAug(aug_src='wordnet', model_path=None, name='Synonym_Aug', aug_min=1, aug_max=10, aug_p=0.3, lang='eng', stopwords=None, tokenizer=None, reverse_tokenizer=None, stopwords_regex=None, force_reload=False, verbose=0)
 |  
 |  Augmenter that leverage semantic meaning to substitute word.
 |  
 |  :param str aug_src: Support 'wordnet' and 'ppdb' .
 |  :param str model_path: Path of dictionary. Mandatory field if using PPDB as data source
 |  :param str lang: Language of your text. Default value is 'eng'. For `wordnet`, you can choose lang from this list
 |      http://compling.hss.ntu.edu.sg/omw/. For `ppdb`, you simply download corresponding langauge pack from
 |      http://paraphrase.org/#/download.
 |  :param float aug_p: Percentage of word will be augmented.
 |  :param int aug_min: Minimum number of word will be augmented.
 |  :param int au